In [28]:
pip install yfinance numpy scipy pandas matplotlib jinja2

Note: you may need to restart the kernel to use updated packages.


In [1]:
from flask import Flask, render_template, request
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
app = Flask(__name__)

In [3]:
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="1d")
    return stock, data

def get_user_input():
    ticker = input("Enter the ticker of the stock: ").upper()
    expiry = input("Enter the expiry date (YYYY-MM-DD): ")
    strike_price = float(input("Enter the strike price: "))
    return ticker, expiry, strike_price


In [4]:
def black_scholes(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:  # put option
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        
    return price

In [5]:
def binomial_tree(S, K, T, r, sigma, steps, option_type="call"):
    # Calculate the time increment
    dt = T / steps
    # Calculate the up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    # Calculate the risk-neutral probability
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Initialize the stock price tree and option price tree
    stock_tree = np.zeros((steps + 1, steps + 1))
    option_tree = np.zeros((steps + 1, steps + 1))
    
    # Stock prices at maturity
    for i in range(steps + 1):
        stock_tree[i, steps] = S * (u ** (steps - i)) * (d ** i)
    
    # Option value at maturity
    for i in range(steps + 1):
        if option_type == "call":
            option_tree[i, steps] = max(0, stock_tree[i, steps] - K)
        elif option_type == "put":
            option_tree[i, steps] = max(0, K - stock_tree[i, steps])
    
    # Backward induction to calculate the option price at each node
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            option_tree[i, j] = np.exp(-r * dt) * (p * option_tree[i, j + 1] + (1 - p) * option_tree[i + 1, j + 1])
    
    return option_tree[0, 0], stock_tree, option_tree

In [6]:
@app.route('/', methods=['GET', 'POST'])
def index():
    bs_price = None
    bt_price = None
    
    if request.method == 'POST':
        ticker = request.form['ticker'].upper()
        expiry = request.form['expiry']
        strike_price = float(request.form['strike_price'])
        steps = int(request.form['steps'])
        
        stock = yf.Ticker(ticker)
        data = stock.history(period="1d")
        S = data['Close'][0]
        r = 0.01  # risk-free rate
        
        # Ensure both dates are timezone-naive
        expiry_date = pd.to_datetime(expiry).date()
        current_date = pd.to_datetime(data.index[0]).date()
        
        T = (expiry_date - current_date).days / 365.0
        sigma = stock.history(period="1y")['Close'].pct_change().std() * np.sqrt(252)
        
        bt_price, stock_tree, option_tree = binomial_tree_complete(S, strike_price, T, r, sigma, steps, option_type="call")
        
        return render_template('index.html', bt_price=bt_price, stock_tree=stock_tree, option_tree=option_tree)
    return render_template('index.html')

In [ ]:
if __name__ == '__main__':
    app.run(use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/Aug/2024 11:48:12] "GET / HTTP/1.1" 200 -
C:\Users\VSK\AppData\Local\Temp\ipykernel_19748\976147001.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S = data['Close'][0]
[2024-08-31 11:48:37,625] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\VSK\anaconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\VSK\anaconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\VSK\anaconda3\Lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self